In [ ]:
#hide 
#Standard Lib imports
import sys, os
from pathlib import Path

# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))

In [ ]:
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
from pprint import pprint as pp
from datetime import datetime as dt
from typing import *
import rfpy
from rfpy.constants import *
from rfpy.parser import *
from rfpy.utils import *
from rfpy.blocks import *
from rfpy.main import export_metadata
from rfpy.parser import _extract_level
from nbdev.showdoc import *
from fastcore.xtras import Path
from fastcore.foundation import L
from fastcore.test import *
from fastcore.parallel import parallel
import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload 2 

In [ ]:
entrada = Path(r'C:\Users\rsilva\Downloads\entrada')
saida = Path(r'C:\Users\rsilva\Downloads\saida')

In [ ]:
arquivos = get_files(entrada) ;  arquivos

(#5) [Path('C:/Users/rsilva/Downloads/entrada/201201_T153421_OneThreadID.bin'),Path('C:/Users/rsilva/Downloads/entrada/201201_T154509_MultiplesThreadID.bin'),Path('C:/Users/rsilva/Downloads/entrada/rfeye002092_210223_T163131_MaskBroken.bin'),Path('C:/Users/rsilva/Downloads/entrada/rfeye002292_210210_T230245_MaskBroken.bin'),Path('C:/Users/rsilva/Downloads/entrada/pmec/rfeye002304_SLMA_PEAK_200828_132002.bin')]

In [ ]:
compressed = arquivos[2]

In [ ]:
%%time
blocks = parse_bin(compressed)

Wall time: 4.53 s


In [ ]:
blocks

{'file_version': 23,
 'string': 'CRFS DATA FILE V023',
 'blocks': defaultdict(fastcore.foundation.L,
             {(21,
               0): (#1) [((36, 111), <rfpy.blocks.DType21 object at 0x000001986935BBC8>)],
              (42,
               0): (#2) [((116, 199), <rfpy.blocks.DType42 object at 0x00000198695A7808>),((204, 339), <rfpy.blocks.DType42 object at 0x00000198695A7208>)],
              (42,
               301): (#1) [((344, 447), <rfpy.blocks.DType42 object at 0x00000198695A71C8>)],
              (42,
               311): (#1) [((452, 551), <rfpy.blocks.DType42 object at 0x00000198695A74C8>)],
              (42,
               321): (#1) [((556, 655), <rfpy.blocks.DType42 object at 0x0000019869587488>)],
              (42,
               331): (#1) [((660, 759), <rfpy.blocks.DType42 object at 0x0000019869587F08>)],
              (68,
               331): (#734) [((764, 955), <rfpy.blocks.DType68 object at 0x00000198695A0EC8>),((5688, 5903), <rfpy.blocks.DType68 object at 0x

In [ ]:
b = blocks['blocks'][(68,311)]

In [ ]:
b

(#11464) [((2792, 3035), <rfpy.blocks.DType68 object at 0x00000198693956C8>),((7816, 8063), <rfpy.blocks.DType68 object at 0x0000019866405048>),((12776, 13023), <rfpy.blocks.DType68 object at 0x00000198695C8B48>),((18944, 19187), <rfpy.blocks.DType68 object at 0x000001987F881A08>),((24696, 24955), <rfpy.blocks.DType68 object at 0x000001987F883A88>),((33696, 33959), <rfpy.blocks.DType68 object at 0x000001987F888248>),((41704, 41939), <rfpy.blocks.DType68 object at 0x000001987F88A408>),((50012, 50255), <rfpy.blocks.DType68 object at 0x000001987F88D588>),((57760, 58003), <rfpy.blocks.DType68 object at 0x000001987F88F748>),((67424, 67663), <rfpy.blocks.DType68 object at 0x000001987F892BC8>)...]

In [ ]:
# %%time
# espectro = extract_block_levels(b)

```c
#define RUN 255 
#define ESC 254 
int run_length_decode(unsigned char *dest, unsigned char *src, int nsrc, int thresh) { 
    int si=0,di=0,nrun; 
    unsigned char ib; 
    while (si < nsrc) { 
        ib = src[si++]; 
        if (ib == RUN) { 
            nrun = src[si++];
            while(nrun-- >0){ 
                dest[di++] = thresh; }
        } else if (ib == ESC) {
            /* next value is literal */ 
            dest[di++] = src[si++];
        } else {
            /* value */ 
            dest[di++] = ib; 
        } 
    } return di; 
} 

int run_length_encode(unsigned char *dest, unsigned char *src, int nsrc, int thresh) { 
    unsigned char ib; 
    int di = 0, si, nunder = 0; 
    for(si = 0; si < nsrc; si++){ 
        ib = src[si]; 
        if ((ib < thresh) && (si < (nsrc-1)) && (nunder < 250)) { 
            nunder += 1; 
        } else { 
            if (nunder > 0) { 
                dest[di++] = RUN; 
                dest[di++] = nunder; 
                nunder = 0; 
            } if ((ib == RUN) || (ib == ESC)) {
                dest[di++] = ESC; 
                dest[di++] = ib; 
            } else { 
                dest[di++] = ib; 
            } 
        } 
    } return di; 
}
```

In [ ]:
%%time
spectrum_blocks = b.itemgot(1)
levels = parallel(_extract_level, spectrum_blocks, n_workers=os.cpu_count(), progress=False)

Wall time: 13.1 s


In [ ]:
b[-1].frequencies

AttributeError: 'tuple' object has no attribute 'frequencies'

In [ ]:
levels[-1]

array([[255., 250.,  67., 255., 250.,  65., 255., 250.,  66., 255., 250.,
         65., 255., 250.,  64., 255., 250.,  63., 255., 250.,  65., 255.,
        250.,  65., 255.,  37.,  96., 118., 124., 119.,  98., 255., 250.,
         65., 255., 250.,  61., 255., 250.,  64., 255., 250.,  65., 255.,
        153., 120., 133., 133., 120., 255., 221., 115., 133., 141., 138.,
        127., 110.,  96.,  68.]], dtype=float16)

In [ ]:
levels[-2]

array([[255., 250.,  65., 255., 250.,  67., 255., 250.,  64., 255., 250.,
         65., 255., 250.,  65., 255., 250.,  63., 255., 250.,  64., 255.,
        250.,  65., 255.,  37.,  98., 119., 125., 119.,  98., 255., 250.,
         63., 255., 250.,  64., 255., 250.,  64., 255., 250.,  66., 255.,
        250.,  65., 255., 127., 104., 134., 148., 148., 135., 101., 255.,
         17.,  98., 255.,  60., 102., 108., 104.,  65.]], dtype=float16)

In [ ]:
levels[0]

array([[255., 148.,  99., 114., 125., 131., 136., 138., 128., 117., 105.,
        255., 250.,  65., 255., 250.,  63., 255., 250.,  63., 255., 250.,
         60., 255., 250.,  61., 255., 250.,  61., 255., 250.,  63., 255.,
        132., 108., 114., 108., 255., 250.,  63., 255., 250.,  61., 255.,
        250.,  61., 255., 250.,  63., 255., 250.,  64., 255., 212.,  98.,
         97.,  62.]], dtype=float16)

In [ ]:
RUN = 255
ESC = 254